In [1]:
import pandas as pd
#from dateutil import parser
import os
from cassandra.cluster import Cluster
import logging 
#from time import time
import shutil

download_dir = "X:\\Data_Analytics\\Equities\\Download\\"
master_dir = "X:\\Data_Analytics\\Equities\\Master\\"
log_path = "X:\\Data_Analytics\\Equities\\"
output_dir = "X:\\Data_Analytics\\Equities\\Output\\"
processed_dir = "X:\\Data_Analytics\\Equities\\Processed_files\\"

# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")



In [2]:
def cassandra_dumper():    
    
    # create python cluster object to connect to your cassandra cluster (specify ip address of nodes to connect within your cluster)
    cluster = Cluster(['127.0.0.1'])
    logging.info('Cassandra Cluster connected...')
    # connect to your keyspace and create a session using which u can execute cql commands 
    session = cluster.connect('test_df')
    logging.info('Using test_df keyspace')
    
    # CREATE A TABLE; dump bhavcopies to this table
    session.execute('CREATE TABLE IF NOT EXISTS equities_bhavcopy (symbol varchar,series varchar,open decimal, high decimal, low decimal, close decimal,last decimal,prevclose decimal,tottrdqty decimal,tottrdval decimal,timestamp date,totaltrades decimal,isin varchar,key varchar, PRIMARY KEY (key, timestamp))')
      
    # walk through all the downlaoded bhavcopies in downlaods dir

    for r,d,f in os.walk(download_dir):

        logging.info('Traverse through each bhavcopy')
        for csv_file in f:
            #read each csv file 
            print csv_file        
            file_name = csv_file
            logging.info('Processing bhavcopy {0}...'.format(file_name))

            bhav_copy = pd.read_csv(download_dir + file_name, keep_default_na=False)   
            # drop unnamed columns
            bhav_copy = bhav_copy.loc[:, ~bhav_copy.columns.str.startswith('Unnamed')]
            # convert to cassandra date format        
            bhav_copy['TIMESTAMP'] = pd.to_datetime(bhav_copy['TIMESTAMP'], format='%d-%b-%Y')
            # concat keys        
            bhav_copy['KEY'] = bhav_copy['SYMBOL']+'_'+bhav_copy['SERIES']

            # write processed bhav_copy to temp file
            bhav_copy.to_csv('temp.csv', index= False)
        
            
            # write csv file to cassandra db
            os.system(log_path+"dump.bat ")   
            # remove bhavcopy from download dir
            logging.info('Removed bhavcopy {0} from download dir'.format(file_name))
            # move to processed folder    
            shutil.copy(download_dir + file_name, processed_dir+file_name)
            
            os.remove(download_dir + file_name)
           
            #os.remove('temp.csv')
            

In [3]:
# walk through all the downlaoded bhavcopies in downlaods dir

for r,d,f in os.walk(download_dir):
    
    logging.info('Traverse through each bhavcopy')
    for csv_file in f:
        #read each csv file 
        print csv_file        
        file_name = csv_file
        logging.info('Processing bhavcopy {0}...'.format(file_name))

        bhav_copy = pd.read_csv(download_dir + file_name, keep_default_na=False)   
        # drop unnamed columns
        bhav_copy = bhav_copy.loc[:, ~bhav_copy.columns.str.startswith('Unnamed')]
        # convert to cassandra date format        
        bhav_copy['TIMESTAMP'] = pd.to_datetime(bhav_copy['TIMESTAMP'], format='%d-%b-%Y')
        # concat keys        
        bhav_copy['KEY'] = bhav_copy['SYMBOL']+'_'+bhav_copy['SERIES']
                                 
        # write processed bhav_copy to temp file
        bhav_copy.to_csv('temp.csv', index= False)
           
        # write csv file to cassandra db
        os.system(log_path+"dump.bat ")   
        # remove bhavcopy from download dir
        logging.info('Removed bhavcopy {0} from download dir'.format(file_name))
        os.remove(download_dir + file_name)
           
        #os.remove('temp.csv')
        

cm19DEC2018bhav.csv
cm20DEC2018bhav.csv
cm21DEC2018bhav.csv
cm24DEC2018bhav.csv


In [5]:
cassandra_dumper()